In [2]:
import torchvision.datasets as datasets
import numpy as np
import torch
import torchvision.transforms as transforms
import os
import json
import PIL
import matplotlib.pyplot as plt

/work3/s183920/02514-DLCI/DLCI-venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/work3/s183920/02514-DLCI/DLCI-venv/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
class TacoDataset(torch.utils.data.Dataset):
    """
    Class to store the food data
    """
    root_dir = '/dtu/datasets1/02514/data_wastedetection/'
    anns_file_path = root_dir + '/' + 'annotations.json'
    
    def __init__(self):
        
        # Read annotations
        with open(self.anns_file_path, 'r') as f:
            self.dataset = json.loads(f.read())
        
        self.categories = self.dataset['categories']
        self.anns = self.dataset['annotations']
        self.imgs = self.dataset['images']
        
        
        self.transform = transforms.Compose([
            transforms.ToTensor(),
        ])
        
    def __getitem__(self, idx):
        img_meta = self.imgs[idx]
        img_ann = self.anns[idx]
        # img = np.load(self.root_dir + img_meta['file_name'])
        img = PIL.Image.open(self.root_dir + img_meta['file_name'])
        img = self.transform(img)
        
        return img, img_meta, img_ann
        # return img_meta

def show_img(img, ax = None):
    if ax is None:
        fig, ax = plt.subplots()
        
    ax.imshow(img.permute(1, 2, 0))
    ax.axis('off')
    
    return ax

dataset = TacoDataset()
img, img_meta, img_ann = dataset.__getitem__(0)

show_img(img)
plt.show()

In [ ]:
def convert_coco_poly_to_mask(segmentations, height, width):
    masks = []
    for polygons in segmentations:
        rles = coco_mask.frPyObjects(polygons, height, width)
        mask = coco_mask.decode(rles)
        if len(mask.shape) < 3:
            mask = mask[..., None]
        mask = torch.as_tensor(mask, dtype=torch.uint8)
        mask = mask.any(dim=2)
        masks.append(mask)
    if masks:
        masks = torch.stack(masks, dim=0)
    else:
        masks = torch.zeros((0, height, width), dtype=torch.uint8)
    return masks

class EncordMaskRCNNDataset(torchvision.datasets.CocoDetection):
   def __init__(self, img_folder, ann_file, transforms=None):
       super().__init__(img_folder, ann_file)
       self._transforms = transforms
 
   def __getitem__(self, idx):
       img, target = super().__getitem__(idx)
       img_metadata = self.coco.loadImgs(self.ids[idx])
 
       image_id = self.ids[idx]
       img_width, img_height = img.size
 
       boxes, labels, area, iscrowd = [], [], [], []
       for target_item in target:
           boxes.append(
               [
                   target_item["bbox"][0],
                   target_item["bbox"][1],
                   target_item["bbox"][0] + target_item["bbox"][2],
                   target_item["bbox"][1] + target_item["bbox"][3],
               ]
           )
 
           labels.append(target_item["category_id"])
           area.append(target_item["bbox"][2] * target_item["bbox"][3])
           iscrowd.append(target_item["iscrowd"])
 
       segmentations = [obj["segmentation"] for obj in target]
       masks = convert_coco_poly_to_mask(segmentations, img_height, img_width)
 
       processed_target = {}
       processed_target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
       processed_target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
       processed_target["masks"] = masks
       processed_target["image_id"] = torch.tensor([image_id])
       processed_target["area"] = torch.tensor(area)
       processed_target["iscrowd"] = torch.as_tensor(iscrowd, dtype=torch.int64)
 
       if self._transforms is not None:
           img, processed_target = self._transforms(img, processed_target)
 
       return img, processed_target, img_metadata
